In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root_scalar
from scipy.integrate import simpson
from scipy import integrate
import matplotlib.pyplot as plt

In [2]:
# define radial sch equation with Z
def fSchrod(En, l, R, Z):
    return l*(l+1.)/R**2-2.*Z/R-En

In [4]:
# define shooting function
def shoot(En, l, Rl, Z):
    def f(r, u):
        return [u[1], fSchrod(En, l, r, Z)*u[0]]
    u0 = [0., 1.]  # initial conditions: u(0)=0, u'(0)=1
    sol = solve_ivp(f, (Rl[0], Rl[-1]), u0, t_eval=Rl, method='RK45', rtol=1e-8)
    ur = sol.y[0]
    return ur[-1]  # return value of u at Rmax
# try shooting for Z=1, l=0, n=1
Z = 1
l = 0
n = 1
Rl = np.linspace(1e-5, 60, 1000)
# find root of shooting function
# For hydrogen atom, E = -Z²/(2n²) in atomic units
E_expected = -Z**2/(2*n**2)
# Set bracket around the expected energy level
sol = root_scalar(shoot, args=(l, Rl, Z), bracket=[1.5*E_expected, 0.5*E_expected], method='bisect', xtol=1e-8)
En = sol.root
print(f'Found energy: E={En:.8f} for Z={Z}, l={l}, n={n}')
# compute solution for found energy
def f(r, u):
    return [u[1], fSchrod(En, l, r, Z)*u[0]]
u0 = [0., 1.]  # initial conditions: u(0)=0, u'(0)=1
sol = solve_ivp(f, (Rl[0], Rl[-1]), u0, t_eval=Rl, method='RK45', rtol=1e-8)
ur = sol.y[0]
# normalize ur
norm = np.sqrt(simpson(ur**2, Rl))
ur /= norm
# plot result
plt.figure(figsize=(8,5))
plt.plot(Rl, ur, label=f'Numerical solution (E={En:.4f})')
plt.plot(Rl, np.exp(-Rl)*Rl*2., '--', label='Analytical solution')
plt.xlim(0,10)
plt.ylim(0,0.5)
plt.xlabel('r')
plt.ylabel('u(r)')
plt.title(f'Hydrogen atom radial wavefunction (Z={Z}, l={l}, n={n})')
plt.grid()
plt.legend()
plt.show()

ValueError: f(a) and f(b) must have different signs